### 1.
In #Chialisp, there are many ways to modularize your code. 

https://chialisp.com/examples#modules

Today we are going to talk about `defmacro`. 🧵

### 2.

In #lisp-based languages (e.g., #clisp #clojure), a macro is a code that generates other code to extend the language. 

#Chialisp also shares a common set of core principles, i.e., treating code as data, and the use of macros by providing `defmacro`.

https://chialisp.com/examples#defmacro

### 3.

At a glance, `defmacro` looks very similar to how function is defined, i.e., `defun`, and `defun-inline`, but it also has pre-define macro operators that can be evaulauted to expand the code during the compile time. 🤯

https://developers.chia.net/t/why-use-macros-instead-of-functions/297

### 4.

How can we use macros to extend the #Chialisp language?

Here is a pleasing `switch` implementation in chialisp by @trepca.

keybase://chat/chia_network.public#chialisp/13357

In [2]:
clsp = Path("switch.clsp").read_text()
map = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(map)

clsp:

   1 (mod input                                                                                                    
   2     (defmacro switch code                                                                                     
   3         (if (r (r code))                                                                                      
   4             (list if (qq (= (unquote (f code)) (unquote (f (f (r code))))))                                   
   5                 (r (f (r code)))                                                                              
   6                 (c switch (c (f code) (r (r code))))                                                          
   7             )                                                                                                 
   8             ; last case is default                                                                            
   9             (r (f (r code)))                                                                                  
  10         )                                                                                                     
  11     )                                                                                                         
  12                                                                                                               
  13     (defun-inline case-a (amount)                                                                             
  14         (* 1000 amount)                                                                                       
  15     )                                                                                                         
  16                                                                                                               
  17     (defun-inline case-b (amount)                                                                             
  18         (* 2000 amount)                                                                                       
  19     )                                                                                                         
  20                                                                                                               
  21     (switch input                                                                                             
  22         ("A" case-a 20)                                                                                       
  23         ("B" case-b 21)                                                                                       
  24         ("default" case-b -1)                                                                                 
  25     )                                                                                                         
  26 )                                                                                                             

clvm:

(a (i (= 1 (q . 65)) (q 1 . 20000) (q 2 (i (= 1 (q . 66)) (q 1 . 0x00a410) (q 1 . -2000)) 1)) 1)                   

### 5.

In addition, being able to generate code by using knowledge at compile time can be useful for creating performant code.

Let's try to create the `sum` function to add four numbers. 

As we want it to be reusable and work with any number of items, we make it a recursive function.

In [11]:
clsp = Path("sum-fun.clsp").read_text()
sum = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(sum)

solution = Program.to([42, 16, 99, 7])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = sum.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(cost)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1 (mod (a b c d)                                                                                                
   2                                                                                                               
   3     (defun sum (items)                                                                                        
   4         (if (r items)                                                                                         
   5             (+ (f items) (sum (r items)))                                                                     
   6             (f items)                                                                                         
   7         )                                                                                                     
   8     )                                                                                                         
   9                                                                                                               
  10     (sum (list a b c d))                                                                                      
  11 )                                                                                                             

clvm:

(a (q 2 2 (c 2 (c (c 5 (c 11 (c 23 (c 47 ())))) ()))) (c (q 2 (i 13 (q 16 9 (a 2 (c 2 (c 13 ())))) (q . 9)) 1) 1)) 

solution:

(42 16 99 7)                                                                                                       

cost:

5768

result:

164                                                                                                                

### 6.
We want to add four numbers together, but the generated CLVM looks complicated and the execution cost is high because the function has to be compiled so that it can call itself recursively.

We can hard-code to make it perform better, but it means we can't reuse the code anymore.

In [6]:
%%bash
brun '(+ 2 5 11 23)' '(42 16 99 7)' -c

cost = 1628
164


### 7.

In this case, we can use `defmacro` to solve performance and reusable issues.

The code looks different, but the generated CLVM is much cleaner and faster. We also can reuse the new macro for any number of items.

The examples show `sum` of 4 and 5 items respectively.

In [15]:
clsp = Path("sum-macro.clsp").read_text()
sum = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(sum)

solution = Program.to([42, 16, 99, 7])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = sum.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(cost)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1 (mod (a b c d)                                                                                                
   2                                                                                                               
   3     (defmacro sum items                                                                                       
   4         (if (r items)                                                                                         
   5             (list                                                                                             
   6                 + (f items)                                                                                   
   7                   (c sum (r items))                                                                           
   8             )                                                                                                 
   9             (f items)                                                                                         
  10         )                                                                                                     
  11     )                                                                                                         
  12                                                                                                               
  13     (sum a b c d)                                                                                             
  14 )                                                                                                             

clvm:

(+ 2 (+ 5 (+ 11 23)))                                                                                              

solution:

(42 16 99 7)                                                                                                       

cost:

2494

result:

164                                                                                                                

### 8.
Let's examine another example which is based on the `map` function example from my previous tweet. 

In this implementation, we define a function inline instead of passing it in.

Look at how clean the generated CLVM comparing to the function one. And the cost is much lower too!

In [16]:
clsp = Path("map-fun.clsp").read_text()
sum = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(sum)

solution = Program.to([10, 20, 30, 40])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = sum.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(cost)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1 (mod (v1 v2 v3 v4)                                                                                            
   2                                                                                                               
   3     ; square function                                                                                         
   4     (defun-inline sqr (v) (* v v))                                                                            
   5                                                                                                               
   6     (defun map (lst)                                                                                          
   7         (if lst                                                                                               
   8             (c                                                                                                
   9                 (sqr (f lst))                                                                                 
  10                 (map (r lst))                                                                                 
  11             )                                                                                                 
  12             ()                                                                                                
  13         )                                                                                                     
  14     )                                                                                                         
  15                                                                                                               
  16     (map (list v1 v2 v3 v4))                                                                                  
  17 )                                                                                                             

clvm:

(a (q 2 2 (c 2 (c (c 5 (c 11 (c 23 (c 47 ())))) ()))) (c (q 2 (i 5 (q 4 (* 9 9) (a 2 (c 2 (c 13 ())))) ()) 1) 1))  

solution:

(>s 20 30 40)                                                                                                      

cost:

8746

result:

(100 400 900 1600)                                                                                                 

In [17]:
clsp = Path("map-macro.clsp").read_text()
sum = Program(compile_clvm_text(clsp, search_paths=["."]))
print("[bold bright_green]clsp:")
print_clsp(clsp)
print("[bold bright_green]clvm:")
print_program(sum)

solution = Program.to([10, 20, 30, 40])
print("[bold bright_green]solution:")
print_program(solution)

cost, result = sum.run_with_cost(INFINITE_COST, solution)
print("[bold bright_green]cost:")
print(cost)
print("[bold bright_green]result:")
print_program(result)

clsp:

   1 (mod (v1 v2 v3 v4)                                                                                            
   2                                                                                                               
   3     ; square function                                                                                         
   4     (defun-inline sqr (v) (* v v))                                                                            
   5                                                                                                               
   6     (defmacro map input                                                                                       
   7         (if (r input)                                                                                         
   8             (list                                                                                             
   9                 c (qq (sqr (unquote (f input))))                                                              
  10                   (c map (r input))                                                                           
  11             )                                                                                                 
  12             (qq (c (sqr (unquote (f input))) ()))                                                             
  13         )                                                                                                     
  14     )                                                                                                         
  15                                                                                                               
  16     (map v1 v2 v3 v4)                                                                                         
  17 )                                                                                                             

clvm:

(c (* 2 2) (c (* 5 5) (c (* 11 11) (c (* 23 23) ()))))                                                             

solution:

(>s 20 30 40)                                                                                                      

cost:

4710

result:

(100 400 900 1600)                                                                                                 

### 9.
Knowing macro can be helpful for defining a new syntax as well as creating a more performant code. 

However, as you can see, it is not straightforward to write a code that generates code and can be more difficult to debug.

https://github.com/Chia-Network/clvm_tools/blob/main/README.md#macros

### 10.
To see more #chialisp macros: 
- https://github.com/Chia-Network/clvm_tools/blob/main/clvm_runtime/macros.clvm
- https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/utility_macros.clib

And here is the notebook:
- https://github.com/kimsk/chia-concepts/blob/main/notebooks/chialisp/chialisp-modules/defmacro-tweets.ipynb

Happy Coding #Chialisp!

### `defmacro` in chialisp
### [Macros](https://github.com/Chia-Network/clvm_tools/blob/main/README.md#macros)
> You can also define macros within a module, which act as inline functions. When a previously defined macro operator is encountered, it "rewrites" the existing statement using the macro, passing along the arguments as literals (ie. they are not evaluated).

- [`qq` and `unquote`](../quote/README.md)
- [defmacro, qq, unquote](https://chialisp.com/examples#defmacro)
- [macros.clvm](https://github.com/Chia-Network/clvm_tools/blob/main/clvm_runtime/macros.clvm)
- [utility_macros.clib](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/utility_macros.clib)

## `switch`
### keybase://chat/chia_network.public#chialisp/13357
> I'm learning macros and while they're quite powerful, it's pretty challenging to write, so interested to get any feedback if this could be written better or especially simpler. 
Here's a "switch" statement implemented as a macro:

```clojure
   (defmacro switch code
        (if (r (r code))
            (list if (qq (= (unquote (f code)) (unquote (f (f (r code))))))
                (r (f (r code)))
                (c switch (c (f code) (r (r code))))
            )
            ; last case is default
            (r (f (r code)))
        )
    )

    (defun-inline case-a (amount)
        (* 1000 amount)
    )

    (defun-inline case-b (amount)
        (* 2000 amount)
    )
 
    (switch input
        ("A" case-a 20) 
        ("B" case-b 21) 
        ("default" case-b -1))
```